In [23]:
# Importing vtk
from vtk import *
import random
from scipy.interpolate import griddata
import numpy as np
import time

## Utility Functions

In [12]:
samplingPercentage = int(input('Enter sampling percentage : ').strip())
interpolationMethod = input('Enter method (linear/nearest):').strip()

Enter sampling percentage : 5
Enter method (linear/neares):nearest


In [14]:
# Getting out polydata from 3d isabel image data

# Importing colors 
colors = vtkNamedColors()

# Initializing XMLImageDataReader object
reader = vtkXMLImageDataReader()

# Setting filename and updating
reader.SetFileName('./Isabel_3D.vti')
reader.Update()

# Taking out ouput from the reader
data = reader.GetOutput()
pressureArray = data.GetPointData().GetArray('Pressure')
numOfPoints = data.GetNumberOfPoints()

(xDim, yDim, zDim) = data.GetDimensions()


In [24]:
def enterDefaultValues():
    a = set({})
    
    t = data.FindPoint(0, 0, 0)
    a.add(t)
    
    t = data.FindPoint(xDim-1, 0, 0)
    a.add(t)

    t = data.FindPoint(xDim-1, yDim-1, 0)
    a.add(t)

    t = data.FindPoint(0, yDim-1, 0)
    a.add(t)
    
    t = data.FindPoint(0, 0, zDim-1)
    a.add(t)

    t = data.FindPoint(xDim-1, 0, zDim-1)
    a.add(t)

    t = data.FindPoint(xDim-1, yDim-1, zDim-1)
    a.add(t)

    t = data.FindPoint(0, yDim-1, zDim-1)
    a.add(t)
    
    return a


def srs(percentage):
    numberOfSampledPoints = int((numOfPoints * percentage)/100)
    indices = enterDefaultValues()
    print(indices)
    numberOfSampledPoints = numberOfSampledPoints
    while(len(indices) < numberOfSampledPoints):
        rp = random.randint(0, numOfPoints-1)
        indices.add(rp)
    return indices
def compute_snr(arrgt, arr_recon):
    diff = arrgt - arr_recon
    sqd_max_diff = (np.max(arrgt)-np.min(arrgt))**2
    snr = 10 * np.log10(sqd_max_diff/np.mean(diff**2))
    return snr

In [25]:
sampledSet = srs(samplingPercentage)

{0, 62499, 3062500, 3124999, 62250, 3124750, 249, 3062749}


In [42]:
sampledPoints = list(sampledSet)
numberOfSampledPoints = len(sampledPoints)
sampledValues = []
sampledCoord = []
for val in sampledPoints:
    sampledValues.append(pressureArray.GetValue(val))
    sampledCoord.append(data.GetPoint(val))

[(0.0, 0.0, 0.0), (145.0, 48.0, 4.0), (48.0, 187.0, 37.0), (3.0, 0.0, 0.0), (5.0, 0.0, 0.0), (187.0, 145.0, 12.0), (91.0, 34.0, 46.0), (15.0, 90.0, 29.0), (17.0, 90.0, 29.0), (193.0, 145.0, 12.0), (164.0, 138.0, 33.0), (200.0, 145.0, 12.0), (28.0, 90.0, 29.0), (173.0, 138.0, 33.0), (165.0, 48.0, 4.0), (105.0, 34.0, 46.0), (241.0, 242.0, 20.0), (205.0, 145.0, 12.0), (166.0, 48.0, 4.0)]


In [27]:
arr = vtkFloatArray()
arr.SetNumberOfValues(numberOfSampledPoints)
for i in range(numberOfSampledPoints):
    arr.SetValue(i, sampledValues[i])
arr.SetName('Pressure')

points = vtkPoints()
for i in range(numberOfSampledPoints):
    points.InsertNextPoint(sampledCoord[i])
    
pdata = vtkPolyData()
pdata.SetPoints(points)
pdata.GetPointData().SetScalars(arr)

0

In [28]:
writer = vtkXMLPolyDataWriter()
writer.SetInputData(pdata)
writer.SetFileName('sample.vtp')
writer.Write()

1

### Reconstruction of Volume Data

In [44]:
allPoints = []
allValues = []
for i in range(numOfPoints):
    allPoints.append(data.GetPoint(i))
    allValues.append(pressureArray.GetValue(val))

if interpolationMethod == 'nearest':
    startTime = time.time()
    reconstructedDataValueList = griddata(sampledCoord, sampledValues, (allPoints), method="nearest")
    endTime = time.time()
else:
    startTime = time.time()
    reconstructedDataValueList = griddata(sampledCoord, sampledValues, (allPoints), method="linear")
    updatedList = []
    updatedCoord = []
    for i in range(numOfPoints):
        if(not (np.isnan(reconstructedDataValueList[i]) == True)):
            updatedList.append(reconstructedDataValueList[i])
            updatedCoord.append(data.GetPoint(i))
    reconstructedDataValueList = griddata(updatedCoord, updatedList, (allPoints), method="nearest")
    endTime = time.time()



3125000


In [45]:
print(len(reconstructedDataValueList))

3125000


In [37]:
reconstructedDataValueVtkArray = vtkFloatArray()
for i in range(numOfPoints):
    reconstructedDataValueVtkArray.InsertNextValue(reconstructedDataValueList[i])
reconstructedImage = vtkImageData()
reconstructedImage.SetDimensions(250, 250, 50)
reconstructedImage.GetPointData().SetScalars(reconstructedDataValueVtkArray)

0

In [46]:
def S_N_R(arr1,arr2):
    diff = arr1-arr2
    sqd_max_diff = (np.max(arr1)-np.min(arr1))**2
    snr = 10*np.log10(sqd_max_diff/np.mean(diff**2))
    return snr

In [47]:
writer = vtkXMLImageDataWriter()
writer.SetFileName("img.vti")
writer.SetInputData(reconstructedImage)
writer.Write()
print("Total time required for interpolation :",endTime - startTime)
print("Signal to Noise Ratio :",S_N_R(np.array(allValues),np.array(reconstructedDataValueList)))

Total time required for interpolation : 5.0525901317596436
Signal to Noise Ratio : -inf


C:\Users\maste\AppData\Local\Temp\ipykernel_3508\3229272073.py:4: RuntimeWarning: divide by zero encountered in log10
  snr = 10*np.log10(sqd_max_diff/np.mean(diff**2))
